# Project Phases

1. Split dataset into train and test.
2. Convert dataset notes into one-hot-encoded classification labels.
3. Convert fingerprint images into 4D Tensors in the form _(nb_samples, 512, 512, 1)_.
4. Model and compile the CNN architecture.
5. Train the model with multiple epochs, using training and validation sets.
6. Load the model with the best validation loss.
7. Compute the accuracy of the model.

## Phase 1: Split dataset into train and test

In [2]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("dataset.csv")

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    df['path'], df['target'], test_size=0.3, random_state=42)

## Phase 2: Convert dataset notes into one-hot-encoded classification labels

In [5]:
label = { 'A': 1, 'T': 2, 'R': 3, 'W': 4, 'L': 0 }
num_classes = len(label)

In [6]:
y_train = [ label[c] for c in y_train ]
y_test = [ label[c] for c in y_test ]

In [7]:
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

## Phase 3: Convert fingerprint images into 4D Tensors

In [8]:
from keras.preprocessing import image

In [9]:
def tensorfy(img_path):
    img = image.load_img(img_path, grayscale=True, target_size=(512, 512))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

In [10]:
def tensorfyList(img_path_list):
    list_of_tensors = [tensorfy(img_path) for img_path in img_path_list]
    return np.vstack(list_of_tensors)

In [11]:
X_train = tensorfyList(X_train)
X_test = tensorfyList(X_test)

## Phase 4: Model and compile the CNN architecture.

In [12]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

In [13]:
model = Sequential()

model.add(Conv2D(16, kernel_size=(3,3), activation='relu', padding='same', input_shape=(512, 512, 1)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model.add(GlobalAveragePooling2D())

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(5, activation='softmax'))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 512, 512, 16)      160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 256, 256, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 64)                0         
__________

In [15]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

## Phase 5: Train the model with multiple epochs, using training and validation sets.

In [16]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

In [17]:
checkpointer = ModelCheckpoint(filepath='best_model.hdf5', 
                               verbose=1, save_best_only=True)

In [18]:
early_stopping = EarlyStopping(monitor='val_loss', patience=8)

In [19]:
model.fit(X_train, y_train,
          validation_split=0.2, epochs=50, verbose=1, batch_size=16,
          callbacks=[checkpointer, early_stopping])

Train on 2240 samples, validate on 560 samples
Epoch 1/50
2240/2240 [==============================] - 29s - loss: 1.8222 - acc: 0.2058 - val_loss: 1.6170 - val_acc: 0.2071
Epoch 2/50
2240/2240 [==============================] - 20s - loss: 1.6006 - acc: 0.2326 - val_loss: 1.5374 - val_acc: 0.3482
Epoch 3/50
2240/2240 [==============================] - 21s - loss: 1.4941 - acc: 0.3469 - val_loss: 1.3898 - val_acc: 0.3804
Epoch 4/50
2240/2240 [==============================] - 21s - loss: 1.3516 - acc: 0.4237 - val_loss: 1.2533 - val_acc: 0.4768
Epoch 5/50
2240/2240 [==============================] - 21s - loss: 1.2685 - acc: 0.4616 - val_loss: 1.1738 - val_acc: 0.5107
Epoch 6/50
2240/2240 [==============================] - 21s - loss: 1.1967 - acc: 0.5036 - val_loss: 1.2039 - val_acc: 0.4661
Epoch 7/50
2240/2240 [==============================] - 21s - loss: 1.1637 - acc: 0.5201 - val_loss: 1.1376 - val_acc: 0.5232
Epoch 8/50
2240/2240 [==============================] - 21s - loss: 1.1

2240/2240 [==============================] - 21s - loss: 0.8732 - acc: 0.6487 - val_loss: 0.8599 - val_acc: 0.6589
Epoch 32/50
2240/2240 [==============================] - 21s - loss: 0.8714 - acc: 0.6482 - val_loss: 0.8714 - val_acc: 0.6482
Epoch 33/50
2240/2240 [==============================] - 21s - loss: 0.8591 - acc: 0.6536 - val_loss: 1.0323 - val_acc: 0.5875
Epoch 34/50
2240/2240 [==============================] - 21s - loss: 0.8593 - acc: 0.6527 - val_loss: 0.9697 - val_acc: 0.5875
Epoch 35/50
2240/2240 [==============================] - 21s - loss: 0.8416 - acc: 0.6616 - val_loss: 0.8631 - val_acc: 0.6589
Epoch 36/50
2240/2240 [==============================] - 21s - loss: 0.8390 - acc: 0.6585 - val_loss: 0.9446 - val_acc: 0.6179
Epoch 37/50
2240/2240 [==============================] - 21s - loss: 0.8497 - acc: 0.6629 - val_loss: 0.8638 - val_acc: 0.6482
Epoch 38/50
2240/2240 [==============================] - 21s - loss: 0.8535 - acc: 0.6487 - val_loss: 0.8207 - val_acc: 0.6

## Phase 6: Load the model with the best validation loss.

In [20]:
model.load_weights('best_model.hdf5')

## Phase 7: Compute the accuracy of the model.

In [21]:
predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in X_test]

In [22]:
test_accuracy = 100 * np.sum(np.array(predictions) == np.argmax(y_test, axis=1)) / len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 61.0000%
